In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from qgsw.fft import dstI1D
from qgsw.fft import dstI1Dffreq
import matplotlib.pyplot as plt

In [ ]:
def laplace(
    f:torch.Tensor,
    dx:float
) -> torch.Tensor:
    return (f[2:]-2*f[1:-1]+f[:-2])/dx**2

n = 100
x = torch.linspace(0,1,n+1) # n+1 pts
dx = x[1]-x[0]
f = sum([i*torch.sin(2**i * torch.pi * x) for i in range(8)]) # n+1 pts
f[-1] = 0

k = torch.pi * torch.arange(1,n) # n-1 pts
eigs = 2 / dx**2 * (torch.cos(torch.arange(1,n)*torch.pi/(n))-1) # n-1 pts

# Laplacian

lap_f = laplace(f,dx) # n+1 pts
lap_f_c = dstI1D(dstI1D(f[1:-1]) * (- k**2))
lap_f_d = dstI1D(dstI1D(f[1:-1]) * (eigs))

plt.figure(figsize=(10,6))
plt.plot(x[1:-1],lap_f, color="black", linewidth=2,linestyle="-", label = "Finite-difference")
plt.plot(x[1:-1],lap_f_d, color="blue", linewidth=2,linestyle="--", label = "Finite-difference spectral laplacian")
plt.plot(x[1:-1],lap_f_c, color="red", linewidth=2,linestyle=":", label = "Continuous spectral laplacian")
plt.legend()
plt.title("Laplacian of f")
plt.show()
plt.figure(figsize=(10,6))
plt.plot(dstI1Dffreq(n-1,dx),dstI1D(lap_f), color="black", linewidth=2,linestyle="-", label = "Finite-difference")
plt.plot(dstI1Dffreq(n-1,dx),dstI1D(lap_f_d), color="blue", linewidth=2,linestyle="--", label = "Finite-difference spectral laplacian")
plt.plot(dstI1Dffreq(n-1,dx),dstI1D(lap_f_c), color="red", linewidth=2,linestyle=":", label = "Continuous spectral laplacian")
plt.legend()
plt.title("Laplacian of f - spectrum")
plt.show()
plt.figure(figsize=(10,6))
plt.plot(x[1:-1],(lap_f-lap_f_d).abs(), color="blue", linewidth=2, label=f"Discrete: {(lap_f-lap_f_d).abs().mean():.5f}")
plt.plot(x[1:-1],(lap_f-lap_f_c).abs(),color="red", linewidth=2, label=f"Continuous: {(lap_f-lap_f_c).abs().mean():.5f}")
plt.legend()
plt.title("Laplacian of f - Absolute error")
plt.show()
plt.figure(figsize=(10,6))
plt.plot(dstI1Dffreq(n-1,dx),(dstI1D(lap_f)-dstI1D(lap_f_d)).abs(), color="blue", linewidth=2, label=f"Discrete: {(dstI1D(lap_f)-dstI1D(lap_f_d)).abs().mean():.5f}")
plt.plot(dstI1Dffreq(n-1,dx),(dstI1D(lap_f)-dstI1D(lap_f_c)).abs(),color="red", linewidth=2, label=f"Continuous: {(dstI1D(lap_f)-dstI1D(lap_f_c)).abs().mean():.5f}")
plt.legend()
plt.title("Laplacian of f - Absolute error spectrums")
plt.show()

# Reconstruction

f_hat = dstI1D(f) # n+1 pts
f_c = torch.nn.functional.pad(dstI1D(dstI1D(laplace(f,dx)) / (- k**2)).real,(1,1))
f_d = torch.nn.functional.pad(dstI1D(dstI1D(laplace(f,dx)) / (eigs)).real,(1,1))

plt.figure(figsize=(10,6))
plt.plot(x,f, color="black", linewidth=2,linestyle="-", label = "True solution")
plt.plot(x,f_d, color="blue", linewidth=2,linestyle="--", label = "Discrete laplacian inversion")
plt.plot(x,f_c, color="red", linewidth=2,linestyle=":", label = "Continuous laplacian inversion")
plt.legend()
plt.title("Reconstruction of f")
plt.show()
plt.figure(figsize=(10,6))
plt.plot(dstI1Dffreq(n+1,dx),dstI1D(f), color="black", linewidth=2,linestyle="-", label = "True solution")
plt.plot(dstI1Dffreq(n+1,dx),dstI1D(f_d), color="blue", linewidth=2,linestyle="--", label = "Discrete laplacian inversion")
plt.plot(dstI1Dffreq(n+1,dx),dstI1D(f_c), color="red", linewidth=2,linestyle=":", label = "Continuous laplacian inversion")
plt.legend()
plt.title("Reconstruction of f - Spectrum")
plt.show()
plt.figure(figsize=(10,6))
plt.plot(x,(f-f_d).abs(), color="blue", linewidth=2, label=f"Discrete: {(f-f_d).abs().mean():.5f}")
plt.plot(x,(f-f_c).abs(),color="red", linewidth=2, label=f"Continuous: {(f-f_c).abs().mean():.5f}")
plt.legend()
plt.title("Reconstruction of f - Absolute errors")
plt.show()
plt.figure(figsize=(10,6))
plt.plot(dstI1Dffreq(n+1,dx),(dstI1D(f)-dstI1D(f_d)).abs(), color="blue", linewidth=2, label=f"Discrete: {(dstI1D(f)-dstI1D(f_d)).abs().mean():.5f}")
plt.plot(dstI1Dffreq(n+1,dx),(dstI1D(f)-dstI1D(f_c)).abs(),color="red", linewidth=2, label=f"Continuous: {(dstI1D(f)-dstI1D(f_c)).abs().mean():.5f}")
plt.legend()
plt.title("Reconstruction of f - Absolute errors spectrums")
plt.show()